In [1]:
#necessary dependencies
import pandas as pd

In [2]:
#overview of data
df = pd.read_csv('For SCL.csv')
df.head()

,DRAINAGE POINT,Date,MonthlyOil (bbl),MonthlyGAS (scf),MonthlyWATER (bbl),CONDENSATE (M3),PROD_DAYS
0,WELL001L-RES1,11/30/1973,18800.81,2966460.0,25.16,NaN,30
1,WELL001L-RES1,12/31/1973,11554.73,1765750.0,12.58,NaN,31
2,WELL001L-RES1,1/31/1974,14385.23,2507365.0,18.87,NaN,31
3,WELL001L-RES1,2/28/1974,19077.57,3884650.0,62.90,NaN,28
4,WELL001L-RES1,3/31/1974,21511.8,4379060.0,0.00,NaN,31


In [3]:
len(df)

3582

In [4]:
len(df['DRAINAGE POINT'].unique())

35

In [5]:
#oilwell with missing production data and their count
df[df['MonthlyOil (bbl)'].isnull()]['DRAINAGE POINT'].value_counts()

WELL010T-RES6    51
Name: DRAINAGE POINT, dtype: int64

In [6]:
#exclusing oil wells without historical production data
v_oil_wells = list(set(df['DRAINAGE POINT'].unique()) - set(df[df['MonthlyOil (bbl)'].isnull()]['DRAINAGE POINT']))
len(v_oil_wells)

34

In [17]:
#oil wells with more than 100 recorded historical entry
dict_ = {}
for i in v_oil_wells:
    if len(df[df['DRAINAGE POINT'] == i]) > 100:
        dict_[i] = len(df[df['DRAINAGE POINT'] == i])

In [19]:
dict_

{'WELL008S-RES5': 185,
 'WELL014L-RES2': 110,
 'WELL005L-RES2': 189,
 'WELL008L-RES2': 185,
 'WELL003L-RES2': 149,
 'WELL001S-RES3': 186,
 'WELL004S-RES5': 116,
 'WELL014S-RES4': 110,
 'WELL003S-RES4': 208,
 'WELL004L-RES6': 167,
 'WELL009L-RES2': 108,
 'WELL005S-RES4': 189,
 'WELL001L-RES1': 236}

In [27]:
# df[df['DRAINAGE POINT'] == 'WELL008S-RES5']['MonthlyOil (bbl)'].value_counts() #== 0

In [28]:
# new_d = {}
# for i in dict_:
#     new_d[i] = len(df[df[df['DRAINAGE POINT'] == i]['MonthlyOil (bbl)'] == 0])

In [32]:
writer = pd.ExcelWriter('oil_wells.xlsx')

for i in dict_:
    df[df['DRAINAGE POINT'] == i].to_excel(writer, sheet_name = i, index=False)

writer.save()

In [73]:
#new dataframe excluding records with erroneous volumes
new_df = df[df['MonthlyOil (bbl)'] != '-']

In [76]:
#converting the volume of oil column to numeric data type
new_df['MonthlyOil (bbl)'] = pd.to_numeric(new_df['MonthlyOil (bbl)'])
new_df['MonthlyOil (bbl)'].dtype

<ipython-input-76-0f26c7d05439>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['MonthlyOil (bbl)'] = pd.to_numeric(new_df['MonthlyOil (bbl)'])


dtype('float64')

In [80]:
#parsing the correct format to the date column
new_df['Date'] = pd.to_datetime(new_df['Date'], format='%m/%d/%Y', errors='coerce')

#grouping oil wells by production date and droping the 'PROD_DAYS' and 'CONDENSATE (M3)' columns 
df_grouped_date = new_df.groupby(by='Date',as_index=False).sum().drop(columns=['PROD_DAYS','CONDENSATE (M3)'])

<ipython-input-80-49801e84db20>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Date'] = pd.to_datetime(new_df['Date'], format='%m/%d/%Y', errors='coerce')


In [82]:
#sort data by date column
final_df = df_grouped_date.sort_values(by='Date')

In [83]:
len(final_df)

236

In [84]:
#export data to be fed into automl
final_df.to_csv('oil_data.csv')